In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv",)

In [3]:
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1514764620,2017-12-31 23:57:00,BTC/USD,13908.73,13913.26,13874.99,13913.26,-0.295570,-0.514960,3.246953,0.163399,0.363161,0.229565,-0.163178,-4.267351e-16
1,1514764560,2017-12-31 23:56:00,BTC/USD,13827.00,13908.69,13827.00,13859.58,-0.303185,-0.684081,0.974031,0.087233,0.089257,2.307135,-0.163178,-4.267351e-16
2,1514764500,2017-12-31 23:55:00,BTC/USD,13825.05,13825.05,13825.05,13825.05,-0.344218,-0.442047,-0.355376,-0.036330,-0.255116,-0.190940,-0.163178,-4.267351e-16
3,1514764440,2017-12-31 23:54:00,BTC/USD,13884.14,13884.14,13823.88,13854.28,-0.020416,0.373551,-0.839658,-0.079503,-0.318178,-0.296745,1.710867,-4.267351e-16
4,1514764380,2017-12-31 23:53:00,BTC/USD,13854.52,13898.70,13840.85,13884.15,0.367259,0.380942,-0.321717,-0.058034,-0.197810,-0.296745,0.298594,-4.267351e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674991,1609459500,2021-01-01 00:05:00,BTC/USD,29021.86,29023.38,28982.33,28999.50,-0.196942,0.012214,0.806634,0.156727,1.550490,0.483496,-0.163178,-3.812808e-01
2674992,1609459440,2021-01-01 00:04:00,BTC/USD,29048.13,29057.73,29035.61,29045.19,-0.141744,0.278150,1.300388,0.340786,1.726775,-0.296745,0.926236,-4.520608e-01
2674993,1609459380,2021-01-01 00:03:00,BTC/USD,29037.68,29069.39,29019.00,29048.13,-0.035411,0.017070,1.437312,0.486452,1.766349,0.604494,-0.163178,-5.749948e-01
2674994,1609459320,2021-01-01 00:02:00,BTC/USD,29069.80,29073.02,29028.14,29035.89,0.084682,-0.075473,1.262523,0.572733,1.641080,-0.296745,-0.048052,-6.576199e-01


In [4]:
predict_data = data.copy().drop(["Open","Close","High","Low", "Symbol"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume USD","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [5]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [6]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
0,1514764380,2017-12-31 23:53:00,0.367259,0.380942,-0.321717,-0.058034,-0.197810,-0.296745,0.298594,-4.267351e-16,0.373551,-0.020416,-0.839658,-0.079503,-0.318178,-0.442047,-0.344218,-0.355376,-0.036330,-0.255116,-0.684081,-0.303185,0.974031,0.087233,0.089257,-0.514960,-0.295570,3.246953,0.163399,0.363161,1
5,1514764080,2017-12-31 23:48:00,0.067075,0.189996,0.531235,0.213927,0.486277,-0.296745,0.303351,-4.267351e-16,-0.000092,-0.131674,0.530705,0.133421,0.341924,0.538370,-0.309888,0.096442,0.063604,0.196784,0.022998,-0.307925,0.074002,-0.026308,-0.051013,-0.013864,0.140478,0.086720,-0.042420,-0.116987,0
10,1514763780,2017-12-31 23:43:00,0.079739,0.496962,0.176112,0.231868,0.178763,-0.296745,-0.155567,-4.267351e-16,-0.496770,0.179185,-0.494481,0.201988,0.124756,0.521257,-0.322627,0.421305,0.251670,0.320053,-0.668974,-0.227993,0.648752,0.220731,0.297869,-0.010254,-0.348619,0.657538,0.273205,0.545667,1
15,1514763480,2017-12-31 23:38:00,-0.061720,0.535950,1.059072,0.533913,0.569550,-0.266360,-0.163178,-4.267351e-16,-0.002251,-0.138692,0.761273,0.452176,0.441947,-0.408320,-0.211435,0.972935,0.439608,0.510920,0.378815,-0.253362,1.541763,0.416904,0.563653,0.378866,-0.041567,0.939382,0.307023,0.341611,0
20,1514763180,2017-12-31 23:33:00,-0.327387,-0.001997,0.159424,0.480053,0.020813,1.105845,-0.163178,-6.727399e-01,-0.395197,-0.260645,0.153960,0.522682,0.166829,0.309211,0.714781,0.547000,0.569268,0.362417,-0.302037,-0.223150,0.363244,0.548028,0.382862,-0.154288,-0.308199,0.911942,0.569766,0.550384,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674965,1609460820,2021-01-01 00:27:00,0.590014,0.087136,-0.845958,-0.955965,-0.036870,-0.296745,0.175856,6.274754e-01,-0.067007,1.235547,-1.119984,-0.958632,-0.054895,-0.240210,0.475204,-1.079694,-0.914725,0.076414,0.114157,1.370075,-0.904186,-0.870496,0.241613,-0.103320,2.129406,-0.567747,-0.892352,0.229824,1
2674970,1609460520,2021-01-01 00:22:00,-0.105216,-0.184985,-1.236472,-0.858640,0.119269,0.416216,-0.163178,5.892400e-01,0.073330,-0.024904,-0.731635,-0.841248,0.206535,-0.154789,0.023824,-0.725001,-0.876723,0.140989,0.073664,-0.164294,-0.509659,-0.881114,0.161733,0.034851,2.994560,-0.917234,-0.932375,0.032842,0
2674975,1609460220,2021-01-01 00:17:00,-0.287596,-0.061275,-0.363714,-0.821848,0.234938,0.073842,-0.163178,4.120796e-01,-0.111944,0.440705,-0.418507,-0.859043,0.170810,0.543953,-0.071406,-0.484481,-0.916955,0.022221,-0.151104,-0.082928,-1.319534,-1.019712,-0.311650,-0.227717,0.736099,-1.454281,-0.929892,-0.092872,0
2674980,1609459920,2021-01-01 00:12:00,-0.123238,0.541444,-0.116417,-0.822243,0.166855,-0.296745,0.905304,3.885019e-01,-0.146590,1.640388,-0.868890,-0.946685,-0.202488,0.056217,1.563090,-0.551666,-0.886675,-0.054885,-0.416947,-0.144609,-0.728193,-0.852356,0.044756,0.028691,-0.183870,-0.515060,-0.774573,0.312876,1


In [7]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.65398021
Iteration 2, loss = 0.64336664
Iteration 3, loss = 0.63996147
Iteration 4, loss = 0.63833888
Iteration 5, loss = 0.63697008
Iteration 6, loss = 0.63595846
Iteration 7, loss = 0.63490123
Iteration 8, loss = 0.63386533
Iteration 9, loss = 0.63308808
Iteration 10, loss = 0.63230100
Iteration 11, loss = 0.63149277
Iteration 12, loss = 0.63073962
Iteration 13, loss = 0.62990224
Iteration 14, loss = 0.62882980
Iteration 15, loss = 0.62831250
Iteration 16, loss = 0.62773880
Iteration 17, loss = 0.62650033
Iteration 18, loss = 0.62577357
Iteration 19, loss = 0.62499218
Iteration 20, loss = 0.62419966
Iteration 21, loss = 0.62347343
Iteration 22, loss = 0.62242316
Iteration 23, loss = 0.62162843
Iteration 24, loss = 0.62097936
Iteration 25, loss = 0.61968942
Iteration 26, loss = 0.61909913
Iteration 27, loss = 0.61817302
Iteration 28, loss = 0.61718370
Iteration 29, loss = 0.61618977
Iteration 30, loss = 0.61551841
Iteration 31, loss = 0.61454742
Iteration 32, los

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 200, 100), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [35]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [13]:
%load_ext tensorboard

In [14]:
!rm -rf ./logs/

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [15]:
shape = (len(predict_data.columns)-3,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
print(train_data.shape, test_data.shape)

(427998, 28) (107000, 28)


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [18]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
model.fit(train_data, train_labels, epochs=30, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/30
13375/13375 [==============================] - 456s 34ms/step - loss: 0.6532 - accuracy: 0.6126 - precision: 0.6121 - recall: 0.6741 - val_loss: 0.6450 - val_accuracy: 0.6218 - val_precision: 0.6481 - val_recall: 0.5781
Epoch 2/30
13375/13375 [==============================] - 459s 34ms/step - loss: 0.6436 - accuracy: 0.6249 - precision: 0.6243 - recall: 0.6806 - val_loss: 0.6397 - val_accuracy: 0.6297 - val_precision: 0.6324 - val_recall: 0.6680
Epoch 3/30
13375/13375 [==============================] - 450s 34ms/step - loss: 0.6408 - accuracy: 0.6279 - precision: 0.6284 - recall: 0.6773 - val_loss: 0.6426 - val_accuracy: 0.6301 - val_precision: 0.6414 - val_recall: 0.6363
Epoch 4/30
13375/13375 [==============================] - 504s 38ms/step - loss: 0.6387 - accuracy: 0.6302 - precision: 0.6309 - recall: 0.6776 - val_loss: 0.6373 - val_accuracy: 0.6308 - val_precision: 0.6322 - val_recall: 0.6737
Epoch 5/30
13375/13375 [==============================] - 477s 36ms/step - l

In [30]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_2022-09-29\assets


In [32]:
loaded_model = tf.keras.models.load_model('./models/1m_200_2000_200_'+time)

In [33]:
test_loss, test_acc = loaded_model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3344/3344 - 21s - loss: 3.0073 - accuracy: 0.6279 - precision: 0.6242 - recall: 0.6940 - 21s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [50]:
test_labels.value_counts()

1    55000
0    52000
Name: Target, dtype: int64

In [24]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [59]:
#preds = np.argmax(probability_model.predict(test_data),axis=1)
preds = np.where(model.predict(test_data)>0.5, 1, 0)

In [60]:
np.unique(preds, return_counts=True)

(array([0, 1]), array([45852, 61148], dtype=int64))

In [61]:
conf = confusion_matrix(test_labels,preds)
conf

array([[29021, 22979],
       [16831, 38169]], dtype=int64)

In [62]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.6329276803629067

In [63]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6242068424151239